In [23]:
import pandas as pd 
import numpy as np 
import datetime as dt 
from pandas_datareader import data as pdr
import yfinance as yfin
from scipy.stats import norm, t
yfin.pdr_override()

In [2]:
### Set up ###

# Import Data
def getData(stocks,start,end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix 

# Portfolio Performance 
def portfolioPerformance(weights, meanReturns, covMatrix, Time): 
    returns = np.sum(meanReturns*weights)*Time 
    stdev = np.sqrt(np.dot(weights.T,np.dot(covMatrix,weights)))*np.sqrt(Time)
    return returns, stdev 

In [6]:
stocks = ['NVDA', 'META','AAPL', 'NFLX', 'AMZN','CRWD','GOOGL','ADBE']
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 100)

returns, meanreturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()

[*********************100%%**********************]  8 of 8 completed


In [12]:
weights = np.random.random(len(returns.columns)) 
weights /= np.sum(weights)

returns['portfolio'] = returns.dot(weights)
returns

Ticker,AAPL,ADBE,AMZN,CRWD,GOOGL,META,NFLX,NVDA,portfolio
Date,,,,,,,,,
2024-02-12,-0.009002,-0.024505,-0.012095,-0.024443,-0.009866,0.001688,-0.006182,0.001594,-0.007166
2024-02-13,-0.011274,-0.016262,-0.021469,0.003579,-0.016200,-0.018725,-0.005969,-0.001661,-0.010924
2024-02-14,-0.004810,0.004602,0.013876,0.037557,0.005512,0.028601,0.044741,0.024567,0.020157
2024-02-15,-0.001575,-0.023517,-0.006901,-0.009475,-0.021721,0.022714,0.024390,-0.016806,-0.001712
2024-02-16,-0.008430,-0.074148,-0.001708,-0.006458,-0.015760,-0.022127,-0.016025,-0.000619,-0.010875
...,...,...,...,...,...,...,...,...,...
2024-05-13,0.017645,0.001700,-0.004854,-0.004926,0.002905,-0.017199,0.009364,0.005797,0.001843
2024-05-14,0.006173,-0.014821,0.002680,0.032489,0.007095,0.008205,-0.004752,0.010586,0.007325
2024-05-15,0.012218,0.019750,-0.005773,0.041177,0.012739,0.020536,-0.000228,0.035838,0.015830


In [21]:
def historicalVaR(returns, alpha=5):
    """ 
    Input: pd df or series of returns
    Output: percentile of distribution at given alpha confidence level 
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    elif isinstance(returns, pd.DataFrame): 
        return returns.aggregate(historicalVaR, alpha = 5)
    else: 
        raise TypeError("Expected returns to be dataframe or series")

#print(historicalVaR(returns, alpha=5))

def historicalCVaR(returns, alpha=5): 
    """ 
    Input: pd df or series of returns 
    Output: conditional value at risk (mean of distribution conditional on being below the value at risk)
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    elif isinstance(returns, pd.DataFrame): 
        return returns.aggregate(historicalCVaR, alpha = 5)
    else: 
        raise TypeError("Expected returns to be dataframe or series")

#print(historicalCVaR(returns, alpha=5))

In [22]:
Time = 1

VaR = historicalVaR(returns['portfolio'],alpha = 5) * np.sqrt(Time)
CVaR = historicalCVaR(returns['portfolio'],alpha = 5) * np.sqrt(Time)
pRet, pStDev = portfolioPerformance(weights,meanreturns,covMatrix,Time)

InitialInvestment = 100000 
print('Expected Portfolio Return:   ',round(InitialInvestment*pRet,2)) # how much we expect to make in a day
print('Value at Risk 95th CI:       ',round(InitialInvestment*VaR,2))
print('Conditional VaR 95th CI:     ',round(InitialInvestment*CVaR,2))

Expected Portfolio Return:    168.23
Value at Risk 95th CI:        -1759.9
Conditional VaR 95th CI:      -2660.22


In [26]:
### Parametric VaR and CVaR ###

def var_parametric(portfolioReturn, portfolioStDev, distribution='normal',alpha=5, dof=7):
    """ 
    Calculate portfolio VaR given a set distribution
    """
    if distribution == 'normal':
        VaR = norm.ppf(1-alpha/100)*portfolioStDev - portfolioReturn
    elif distribution == "t-distribution":
        # nu := dof, alpha, h := lookforward time period
        nu = dof 
        VaR = np.sqrt((nu-2)/nu)*t.ppf(1-alpha/100,nu)*portfolioStDev - portfolioReturn
    else: 
        raise TypeError("Expected distribution to be 'normal' or 't-distribution'")
    return VaR 

def cvar_parametric(portfolioReturn, portfolioStDev, distribution='normal',alpha=5, dof=7):
    """ 
    Calculate portfolio CVaR given a set distribution
    """
    if distribution == 'normal':
        CVaR = (alpha/100)**-1 * norm.pdf(norm.ppf(alpha/100))*portfolioStDev - portfolioReturn 
    elif distribution == "t-distribution":
        # nu := dof, alpha, h := lookforward time period
        nu = dof 
        x_anu = t.ppf(alpha/100, nu)
        CVaR = -(alpha/100)**-1 * (1-nu)**-1 * (nu - 2 + x_anu**2) * t.pdf(x_anu, nu) * portfolioStDev - portfolioReturn
    else: 
        raise TypeError("Expected distribution to be 'normal' or 't-distribution'")
    return CVaR 

normVaR = var_parametric(pRet,pStDev)
normCVaR = cvar_parametric(pRet,pStDev)
tVaR = var_parametric(pRet,pStDev,distribution="t-distribution")
tCVaR = cvar_parametric(pRet,pStDev,distribution="t-distribution")

print('Normal VaR:    ',round(InitialInvestment*normVaR,2)) # how much we expect to lose in the worst 5% given a normal distribution
print('Normal CVaR:   ',round(InitialInvestment*normCVaR,2))
print('T-dist VaR:    ',round(InitialInvestment*tVaR,2))
print('T-dist CVaR:   ',round(InitialInvestment*tCVaR,2))

Normal VaR:     2362.86
Normal CVaR:    3005.86
T-dist VaR:     2295.71
T-dist CVaR:    3070.52
